In [20]:
import pandas as pd
import numpy as np
import pickle 
import _pickle as cPickle
import time
import boto3
import gc
import os

In [2]:
chapterLabelDF = pd.read_csv("chapter_label.csv").astype({'HADM_ID': 'str'})
chapterLabelDF.columns

Index(['HADM_ID', '001_139', '140_239', '240_279', '280_289', '290_319',
       '320_389', '390_459', '460_519', '520_579', '580_629',
       '630_679__740_759__760_779', '680_709', '710_739', '780_799', '800_999',
       'E_V'],
      dtype='object')

In [3]:
chapterLabelsHADMMap=None

In [4]:
chapterLabelsHADMMap = {}
for col in chapterLabelDF.columns[1:]:
    #print(col)
    chapterLabelsHADMMap[col]=chapterLabelDF.loc[chapterLabelDF[col]==1,'HADM_ID']
    
print("Chapter : # of HADM_ID")
for key in chapterLabelsHADMMap.keys():
    print(key,":",len(chapterLabelsHADMMap[key]))
chapterLabelDF = None

Chapter : # of HADM_ID
001_139 : 14535
140_239 : 8912
240_279 : 35606
280_289 : 19366
290_319 : 16245
320_389 : 15828
390_459 : 42650
460_519 : 25260
520_579 : 21010
580_629 : 21779
630_679__740_759__760_779 : 7619
680_709 : 6076
710_739 : 10053
780_799 : 20182
800_999 : 22686
E_V : 41887


In [5]:
# HADMChapterMap = {}
# #chapterLabelDF_Small = chapterLabelDF.iloc[0:5,]
# count=0
# for hadm in chapterLabelDF.loc[:,'HADM_ID']:
#     count = count+1
#     if count%10000==0 :
#         print("Processed", count)
#     list_of_chapters = []
#     for col in chapterLabelDF.columns[1:]:
#         #print(col)
#         #print(hadm)
#         #print(chapterLabelDF_Small.loc[chapterLabelDF_Small['HADM_ID']==hadm,col]==1)
#         if (chapterLabelDF.loc[chapterLabelDF['HADM_ID']==hadm,col]==1).any():
#             #list_of_chapters = HADMChapterMap.get(hadm,[])
#             list_of_chapters.append(col)
#     HADMChapterMap[hadm]=list_of_chapters
    

In [6]:
#len(list(HADMChapterMap.items()))

In [18]:
def loadFromS3(embeddingFileName):
    # https://w210-mimic.s3.amazonaws.com/embeddings1
    now = time.time()
    print("Starting Load for File",embeddingFileName)
    embeddings = {}
    session = boto3.session.Session(region_name='us-east-1')
    s3client = session.client('s3')
    response = s3client.get_object(Bucket='w210-mimic', Key=embeddingFileName)
    body_string = response['Body'].read()
    embeddings.update(cPickle.loads(body_string))
    with open("embeddings/"+embeddingFileName, 'wb') as handle:
        pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("File Loaded in:", time.time()-now)
    
    return embeddings

def LoadFromDisk(embeddingFileName, directory="embeddings"):
    embeddingFileHandle = open(directory+"/"+embeddingFileName, 'rb')
    return embeddingFileHandle

def LoadDataFromDisk(embeddingFileName, directory="embeddings"):
    with  open(directory+"/"+embeddingFileName, 'rb') as embeddingFileHandle:
        data = np.load(embeddingFileHandle,allow_pickle=True)
    return data

def SaveToS3(file, bucket='w210-mimic', path="embeddings_per_chapter"):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = path + '/' + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)



In [8]:
gc.enable()
gc.collect()

5

In [9]:
#embeddingFile='embeddings2'
#data=None
#print(embeddingFile)
# dataHandle =  LoadFromDisk(embeddingFile)
# print(dataHandle)
# data = np.load(dataHandle,allow_pickle=True)
#data = LoadDataFromDisk(embeddingFile)
#data = cPickle.loads(dataHandle.read())

In [10]:
#len(data.items())
#data=None

In [11]:
'124064' in set(chapterLabelsHADMMap['001_139'])

True

In [12]:
#list(HADMChapterMap.items())[0:5]

In [21]:
dir_to_save_embeddings_per_chapter="embeddings_per_chapter/"

#'embeddings6','embeddings1', 'embeddings2', 'embeddings3', 'embeddings4', 'embeddings5',
for embeddingFile in [ 'embeddings1']:
    print('Processing file=',embeddingFile, time.time())
    data = None
    gc.collect()
    data = LoadDataFromDisk(embeddingFile)
    print("Data loaded", time.time())
    
    for chapter in chapterLabelsHADMMap.keys():
        print('On chapter=',chapter, time.time())
        hadms_in_chpater = set(chapterLabelsHADMMap[chapter])        
        hadms_in_data = set(list(data.keys()))
        
        hadm_intersection_chapter = hadms_in_data.intersection(hadms_in_chpater) 
        print('HADM in chapter=',len(hadms_in_chpater),'HADM in Data=',len(hadms_in_data),'HADM in intersection=',len(hadm_intersection_chapter))
        
        filename_to_save_to = dir_to_save_embeddings_per_chapter + embeddingFile + "_" + chapter
        print('filename_to_save_to=',filename_to_save_to)    
        with open(filename_to_save_to, "wb") as filetoWrite: 
            for hadm in hadm_intersection_chapter:
                embeddings_per_chapter = {}
                embeddings_per_chapter[hadm] = data[hadm]
                pickle.dump(embeddings_per_chapter, filetoWrite,protocol=pickle.HIGHEST_PROTOCOL) 
        
#         print("About to save chapter level embeddings to file", time.time())
#         # save embeddings per chapter in different file            
#         filename_to_save_to = embeddingFile + "_" + chapter
#         print('filename_to_save_to=',filename_to_save_to)
#         with open(dir_to_save_embeddings_per_chapter + filename_to_save_to, "wb") as filetoWrite: 
#             pickle.dump(embeddings_per_chapter, filetoWrite,protocol=pickle.HIGHEST_PROTOCOL) 
#         embeddings_per_chapter = None
        print('Moving file ', filename_to_save_to, ' to S3')
        SaveToS3(filename_to_save_to)
        os.remove(filename_to_save_to)
        gc.collect()
            
    data=None
    gc.collect()
    

Processing file= embeddings1 1594816737.4592671
Data loaded 1594816941.642603
On chapter= 001_139 1594816941.6430156
HADM in chapter= 14535 HADM in Data= 10000 HADM in intersection= 3079
filename_to_save_to= embeddings_per_chapter/embeddings1_001_139
Moving file  embeddings_per_chapter/embeddings1_001_139  to S3


ClientError: An error occurred (EntityTooLarge) when calling the PutObject operation: Your proposed upload exceeds the maximum allowed size

In [16]:
def unpickle_iter(file):
    try:
        while True:
             yield cPickle.load(file)
    except EOFError:
        raise StopIteration
count=0        
with open('embeddings_per_chapter/embeddings2_140_239',"rb") as file:
    for item in unpickle_iter(file):
        print(item)
        count = count+1
        if count>10:
            break

{'166349': array([[[ 0.118   ,  0.3936  , -0.305   , ..., -0.014145, -0.1495  ,
         -0.186   ],
        [-0.3833  , -0.0584  ,  0.5503  , ...,  0.0145  , -0.1682  ,
          0.3113  ],
        [-0.1952  ,  0.1453  ,  0.375   , ...,  0.3333  , -0.2108  ,
          0.003254],
        ...,
        [-0.192   , -0.4229  , -0.1763  , ...,  0.622   , -0.462   ,
         -0.2712  ],
        [-0.1646  , -0.2268  , -0.10767 , ...,  0.5044  , -0.334   ,
         -0.2367  ],
        [ 0.38    , -0.04047 , -0.3027  , ...,  0.5347  ,  0.08716 ,
         -0.6406  ]],

       [[ 0.1934  ,  0.341   , -0.1696  , ..., -0.0797  ,  0.169   ,
         -0.2505  ],
        [-0.3098  , -0.0428  , -0.0988  , ...,  0.3257  ,  0.4731  ,
          0.049   ],
        [-0.3318  ,  0.2327  ,  0.3708  , ...,  0.08386 ,  0.2878  ,
         -0.01825 ],
        ...,
        [-0.3674  , -0.0814  , -0.008224, ..., -0.2083  , -0.321   ,
         -0.0319  ],
        [-0.2444  , -0.1897  ,  0.01634 , ..., -0.1833  , -0.

In [ ]:
dir_to_save_embeddings_per_chapter="embeddings_per_chapter/"

#'embeddings1', 'embeddings2', 'embeddings3', 'embeddings4', 'embeddings5',
for embeddingFile in [ 'embeddings6','embeddings1']:
    print('Processing file=',embeddingFile)
    #data = LoadFromS3(embeddingFile)
    data=None
    dataHandle =  LoadFromDisk(embeddingFile)
    data = np.load(dataHandle,allow_pickle=True)
    embeddings_per_chapter = {}    
    for hadm, embeddings in list(data.items())[0:5]:
        print('hadm=',hadm)
        print('len(embeddings)=',len(embeddings))
        if hadm in HADMChapterMap:
            chapter_list = HADMChapterMap[hadm]
            print('chapter_list=',chapter_list)
            for chapter in chapter_list:
                print('chapter=',chapter)
                embeddings_for_cur_chapter = embeddings_per_chapter.get(chapter,{})
                embeddings_for_cur_chapter[hadm] = embeddings
                embeddings_per_chapter[chapter] = embeddings_for_cur_chapter
    # save embeddings per chapter in different file            
    for chapter in embeddings_per_chapter.keys():
        embeddings_to_save = embeddings_per_chapter[chapter]
        filename_to_save_to = embeddingFile + "_" + chapter
        print('filename_to_save_to=',filename_to_save_to)
        with open(dir_to_save_embeddings_per_chapter + filename_to_save_to, "wb") as filetoWrite: 
            pickle.dump(embeddings_to_save, filetoWrite,protocol=pickle.HIGHEST_PROTOCOL) 

In [ ]:
with open("embeddings_per_chapter/embeddings6_520_579", "rb") as fileToRead: 
        f=pickle.load(fileToRead)
        
f

In [ ]:
lineCount=0
with open("embeddings/embeddings6.4.1", "ab") as filetoWrite: 
    with open("embeddings/embeddings6", "rb") as fileToRead:
        while True:
            lines = fileToRead.readlines(100000)
            if lines is None:
                break;

            pickle.dump(lines, filetoWrite, protocol=4)

        